This jupyter notebook is here to test properties of the training set (positive-defineteness, symmetry, etc)

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
from torch.nn import functional as F
from os.path import exists
import numpy as np
import time, math, sys
import matplotlib.pyplot as plt
import matplotlib.colors as colors
plt.rcParams['figure.facecolor'] = 'white'

from CovNet import Network_Full, Network_ReverseVGG, MatrixDataset, symmetric_log, symmetric_exp, matrix_loss

In [2]:
train_log = False
train_inverse = False
N = 52500
training_dir = "/home/joeadamo/Research/Data/Training-Set/"
save_dir = "/home/joeadamo/Research/CovA-NN-Emulator/Plots/"
BOSS_dir = "/home/joeadamo/Research/Data/BOSS-DR12/"
CovaPT_dir = "/home/joeadamo/Research/CovaPT/Example-Data/"

In [3]:
t1 = time.time()
data = MatrixDataset(training_dir, N, 0, train_log, train_inverse)
t2 = time.time()
print("Done loading in data, took {:0.2f} s".format(t2 - t1))

Done loading in data, took 231.63 s


In [5]:
non_positive_diag = []
non_symmetric = []
non_definete = []
t1 = time.time()

min_asym = 10000; max_asym = 0; avg_asym = 0.
# first test the base matrix
for i in range(N):
    C = data[i][1].detach().numpy()
    
    # test diagonal is all positives
    for i in range(C.shape[0]):
        if C[i,i] <= 0:
            non_positive_diag.append(i)
            break
    
    # test assymetry
    C_sym = 0.5*(C + C.T)
    C_asym = 0.5*(C - C.T)
    symmetric_val = np.linalg.norm(C_asym, 2) / np.linalg.norm(C_sym, 2)
    if abs(symmetric_val) > 1e-14: # this number should be 0 if the matrix is symmetric
        non_symmetric.append(i)
        min_asym = min(symmetric_val, min_asym); max_asym = max(symmetric_val, max_asym)
        avg_asym += symmetric_val
    
    # test eigenvalues are all positive
    eigen, v = np.linalg.eig(C)
    if np.all(eigen > 0.) == False:
        non_definete.append(i)
        
t2 = time.time()
print("Done! took {:0.0f} minutes {:0.2f} seconds".format(math.floor((t2 - t1)/60), (t2 - t1)%60))
print(len(non_positive_diag), "Matrices have non-positive diagonals")
print(len(non_symmetric), "({:0.2f}%) Matrices are assymetric".format(100.*len(non_symmetric)/N))
print("min / max symmetric val: [{:0.3e}, {:0.3e}] (avg = {:0.3e}".format(min_asym, max_asym, avg_asym / len(non_symmetric)))
print(len(non_definete), "Matrices don't have all positive eigenvalues")

Done! took 4 minutes 33.79 seconds
0 Matrices have non-positive diagonals
11200 (21.33%) Matrices are assymetric
min / max symmetric val: [1.000e-14, 5.899e-13] (avg = 5.135e-14
0 Matrices don't have all positive eigenvalues


In [6]:
# now test the inverse matrix

non_positive_diag_inv = []
non_symmetric_inv = []
non_definete_inv = []
min_asym = 10000; max_asym = 0; avg_asym = 0.
t1 = time.time()
for i in range(N):
    C = data[i][1].detach().numpy()
    P = np.linalg.inv(C)
    
    # test diagonal is all positives
    for i in range(C.shape[0]):
        if P[i,i] <= 0:
            non_positive_diag_inv.append(i)
            break
    
    # test assymetry
    P_sym = 0.5*(P + P.T)
    P_asym = 0.5*(P - P.T)
    symmetric_val = np.linalg.norm(P_asym, 2) / np.linalg.norm(P_sym, 2)
    if abs(symmetric_val) > 1e-14: # this number should be 0 if the matrix is symmetric
        non_symmetric_inv.append(i)
        min_asym = min(symmetric_val, min_asym); max_asym = max(symmetric_val, max_asym)
        avg_asym += symmetric_val
    
    # test eigenvalues are all positive
    eigen, v = np.linalg.eig(P)
    if np.all(eigen > 0.) == False:
        non_definete_inv.append(i)
        
t2 = time.time()
print("Done! took {:0.0f} minutes {:0.2f} seconds".format(math.floor((t2 - t1)/60), (t2 - t1)%60))
print(len(non_positive_diag_inv), "Matrices have non-positive diagonals")
print(len(non_symmetric_inv), "({:0.2f}%) Matrices are assymetric".format(100.*len(non_symmetric_inv)/N))
print("min / max symmetric val: [{:0.3e}, {:0.3e}] (avg = {:0.3e}".format(min_asym, max_asym, avg_asym / len(non_symmetric)))
print(len(non_definete_inv), "Matrices don't have all positive eigenvalues")

Done! took 5 minutes 32.72 seconds
0 Matrices have non-positive diagonals
17681 (33.68%) Matrices are assymetric
min / max symmetric val: [1.001e-14, 5.529e-09] (avg = 1.152e-12
0 Matrices don't have all positive eigenvalues
